In [2]:
########################################################################
# II. PREDICT AVERAGE SESSION LENGTH
########################################################################
import pandas as pd
import pickle
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.model_selection import *



In [8]:
# method to find IP address class
def find_class(ip):
    a=int(ip.split('.')[0])
    
  
    # Class A 
    if (a >=1 and a <= 126) :
        return 1 
  
    # Class B 
    elif (a >= 128 and a <= 191): 
        return 2
  
    # Class C 
    elif (a >= 192 and a <= 223) :
        return 3 
  
    # Class D 
    elif (a >= 224 and a <= 239) :
        return 4 
    # Class E
    else:
        return 5 


In [9]:

# Read output of intervals output from DE section

intervals = pd.read_csv('/Users/bjayaram/Downloads/WeblogChallenge-master/data/output/intervals/part-00000-26ee749b-0fdb-49ca-a734-f9c31ca4842e-c000.csv',
    low_memory=False)
intervals.columns


Index([u'session_id', u'ip', u'max(ts)', u'min(ts)', u'interval_seconds'], dtype='object')

In [10]:

# calculate total average interval across sessions for each IP
data = intervals[['session_id', 'ip', 'interval_seconds']].groupby('ip').mean()
data['ip'] = data.index
data.reset_index(drop=True, inplace=True)


In [11]:
# Split IP address into 4 features
X = data['ip'].str.split('.', expand=True)

for i in range(4):
    c = chr(ord('a') + i)
    X[c] = map(int, X[i])
    
# Add session id
X['session_id'] = data['session_id']

# find class (i.e A, B, C, D) for each IP address. Method find_class() defined in parse_ip.py
X['class'] = map(find_class, data['ip'])


In [13]:
#  X contains IP class, session id, 4 components of IP address
X = X[['class', 'session_id', 'a', 'b', 'c', 'd']].values

# Target variable Y contains session length in minutes
Y = (data['interval_seconds']/60).values

In [14]:
alpha_list = range(10,10000,10)
alphas = map(lambda x: 10.0/(1+x), alpha_list)
results={}


# Try the following regressors: 
# 1.Linear Regression, 
# 2.GradientBoostingRegressor, 
# 3.Ridge, 
# 4.Lasso, 
# 5.RandomForest
model1 = GradientBoostingRegressor(n_estimators=1000)
model2 = RidgeCV(normalize=True, alphas = alphas)
model3 = LassoCV(normalize=True, alphas = alphas, max_iter=1000)
model4 = RandomForestRegressor(n_estimators=1000)
model5 = LinearRegression(normalize=True)


In [15]:
# Run 10-fold cross validation on each of the models to evaluate the performance
results1 = cross_val_score(model1, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')
results2 = cross_val_score(model2, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')
results3 = cross_val_score(model3, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')
results4 = cross_val_score(model4, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')
results5 = cross_val_score(model5, X, Y, cv=10, n_jobs=-1,scoring='neg_mean_squared_error')
results['Gradient Boosting Regression']=results1.mean()
results['Ridge Regresison']=results2.mean()
results['Lasso Regression']=results3.mean()
results['RandomForestRegressor']=results4.mean()
results['Linear Regression']=results5.mean()


In [ ]:
print results
# -ve MSE (in minutes) values are as follows:
# {'Ridge Regresison': -24.683386466253015,
#  'Lasso Regression': -24.669352663188963,
#  'Linear Regression': -24.683337971018613,
#  'Gradient Boosting Regression': -25.380661544821258,
#  'RandomForestRegressor': -26.480090281772327}

In [ ]:
# Further iterate with lasso


alphas=map(lambda x:pow(10,x),range(-100,100,10))
model=GridSearchCV(Lasso(),n_jobs=-1, cv=10,scoring='neg_mean_squared_error',verbose=1,
                   param_grid={'alpha':alphas,'normalize':[True],'max_iter':[100,1000,10000]})
model.fit(X,Y)
print "BEST SCORE", model.best_score_, model.scoring
print "BEST ESTIMATOR", model.best_estimator_
pickle.dump(model,open('/Users/bjayaram/Downloads/WeblogChallenge-master/WebLogsPrediction/output/intervals_model.p','w'))
# The final MSE value is 24.669352663188963 minutes

ValueError: n_components=4 must be between 0 and n_features=2 with svd_solver='full'

NameError: name 'train' is not defined

In [ ]:


df['date']=df['ts'].dt.date
df['hour']=df['ts'].dt.hour
df['minute']=df['ts'].dt.minute
df['second']=df['ts'].dt.second

requests=df[time_fields+['ts']]
# requests.to_csv('/Users/bjayaram/Downloads/WeblogChallenge-master/WebLogsPrediction/output/requests.csv')
# time_fields=['date','hour','minute','second']
# requests = requests.groupby(time_fields).count()
# requests.reset_index()

In [53]:
requests.reset_index(level=0, inplace=True)
requests.groupby(['date','hour','minute','second'])

,index,date,hour,minute,second,ts
0,0,2015-07-22,9,0,28,2015-07-22 09:00:28.019143
1,1,2015-07-22,9,0,27,2015-07-22 09:00:27.894580
2,2,2015-07-22,9,0,27,2015-07-22 09:00:27.885745
3,3,2015-07-22,9,0,28,2015-07-22 09:00:28.048369
4,4,2015-07-22,9,0,28,2015-07-22 09:00:28.036251
5,5,2015-07-22,9,0,28,2015-07-22 09:00:28.033793
6,6,2015-07-22,9,0,28,2015-07-22 09:00:28.055029
7,7,2015-07-22,9,0,28,2015-07-22 09:00:28.050298
8,8,2015-07-22,9,0,28,2015-07-22 09:00:28.059081
9,9,2015-07-22,9,0,28,2015-07-22 09:00:28.054939
